## CONV-LSTMAE

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-08-24 10:38:06.419496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-24 10:38:06.419559: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [4]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


Conv_LstmAE

In [5]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Conv_LstmAE_HYP.csv')


input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_test =  {}
source_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = Conv_LstmAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_Conv_LstmAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_Conv_LstmAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

model = Conv_LstmAEModel(layer1=40, layer2=20, encoding_dim=2)
model.fit(train_merge_df, train_merge_df, epochs=200, batch_size=256, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_Conv_LstmAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_Conv_LstmAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

2023-08-14 22:24:51.699872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-14 22:24:51.703520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-08-14 22:24:51.703613: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-08-14 22:24:51.703686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-08-14 22:24:51.750471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 8)]            0         
                                                                 
 conv1d (Conv1D)             (None, 1, 40)             1000      
                                                                 
 lstm (LSTM)                 (None, 1, 30)             8520      
                                                                 
 lstm_1 (LSTM)               (None, 1, 2)              264       
                                                                 
 lstm_2 (LSTM)               (None, 1, 30)             3960      
                                                                 
 conv1d_transpose (Conv1DTra  (None, 1, 40)            3640      
 nspose)                                                         
                                                             

70/70 - 0s - loss: 0.0042 - val_loss: 0.0061 - 424ms/epoch - 6ms/step
Epoch 83/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0059 - 424ms/epoch - 6ms/step
Epoch 84/100
70/70 - 0s - loss: 0.0043 - val_loss: 0.0061 - 427ms/epoch - 6ms/step
Epoch 85/100
70/70 - 0s - loss: 0.0042 - val_loss: 0.0062 - 483ms/epoch - 7ms/step
Epoch 86/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0058 - 581ms/epoch - 8ms/step
Epoch 87/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0058 - 568ms/epoch - 8ms/step
Epoch 88/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0062 - 575ms/epoch - 8ms/step
Epoch 89/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0062 - 598ms/epoch - 9ms/step
Epoch 90/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0060 - 603ms/epoch - 9ms/step
Epoch 91/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0062 - 554ms/epoch - 8ms/step
Epoch 92/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0061 - 611ms/epoch - 9ms/step
Epoch 93/100
70/70 - 1s - loss: 0.0042 - val_loss: 0.0060 - 531ms/epoch - 8ms/step
Epoch 94/100
70/7

Epoch 63/100
35/35 - 0s - loss: 0.0030 - val_loss: 0.0042 - 378ms/epoch - 11ms/step
Epoch 64/100
35/35 - 0s - loss: 0.0029 - val_loss: 0.0039 - 390ms/epoch - 11ms/step
Epoch 65/100
35/35 - 0s - loss: 0.0029 - val_loss: 0.0039 - 384ms/epoch - 11ms/step
Epoch 66/100
35/35 - 0s - loss: 0.0028 - val_loss: 0.0039 - 382ms/epoch - 11ms/step
Epoch 67/100
35/35 - 0s - loss: 0.0028 - val_loss: 0.0035 - 393ms/epoch - 11ms/step
Epoch 68/100
35/35 - 0s - loss: 0.0027 - val_loss: 0.0035 - 382ms/epoch - 11ms/step
Epoch 69/100
35/35 - 0s - loss: 0.0027 - val_loss: 0.0037 - 380ms/epoch - 11ms/step
Epoch 70/100
35/35 - 0s - loss: 0.0027 - val_loss: 0.0037 - 352ms/epoch - 10ms/step
Epoch 71/100
35/35 - 0s - loss: 0.0027 - val_loss: 0.0035 - 359ms/epoch - 10ms/step
Epoch 72/100
35/35 - 0s - loss: 0.0026 - val_loss: 0.0038 - 419ms/epoch - 12ms/step
Epoch 73/100
35/35 - 0s - loss: 0.0026 - val_loss: 0.0036 - 377ms/epoch - 11ms/step
Epoch 74/100
35/35 - 0s - loss: 0.0026 - val_loss: 0.0033 - 390ms/epoch - 11

Epoch 44/100
35/35 - 0s - loss: 0.0052 - val_loss: 0.0072 - 309ms/epoch - 9ms/step
Epoch 45/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0071 - 286ms/epoch - 8ms/step
Epoch 46/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0071 - 275ms/epoch - 8ms/step
Epoch 47/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0074 - 294ms/epoch - 8ms/step
Epoch 48/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0071 - 298ms/epoch - 9ms/step
Epoch 49/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0076 - 286ms/epoch - 8ms/step
Epoch 50/100
35/35 - 0s - loss: 0.0051 - val_loss: 0.0071 - 301ms/epoch - 9ms/step
Epoch 51/100
35/35 - 0s - loss: 0.0050 - val_loss: 0.0070 - 291ms/epoch - 8ms/step
Epoch 52/100
35/35 - 0s - loss: 0.0050 - val_loss: 0.0070 - 264ms/epoch - 8ms/step
Epoch 53/100
35/35 - 0s - loss: 0.0050 - val_loss: 0.0070 - 279ms/epoch - 8ms/step
Epoch 54/100
35/35 - 0s - loss: 0.0050 - val_loss: 0.0073 - 281ms/epoch - 8ms/step
Epoch 55/100
35/35 - 0s - loss: 0.0050 - val_loss: 0.0071 - 280ms/epoch - 8ms/step
Epoc

...

Epoch 33/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0060 - 304ms/epoch - 9ms/step
Epoch 34/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0059 - 322ms/epoch - 9ms/step
Epoch 35/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0061 - 325ms/epoch - 9ms/step
Epoch 36/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0058 - 338ms/epoch - 10ms/step
Epoch 37/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0061 - 316ms/epoch - 9ms/step
Epoch 38/100
35/35 - 0s - loss: 0.0047 - val_loss: 0.0060 - 328ms/epoch - 9ms/step
Epoch 39/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0059 - 323ms/epoch - 9ms/step
Epoch 40/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0059 - 358ms/epoch - 10ms/step
Epoch 41/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0057 - 322ms/epoch - 9ms/step
Epoch 42/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0061 - 310ms/epoch - 9ms/step
Epoch 43/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0057 - 326ms/epoch - 9ms/step
Epoch 44/100
35/35 - 0s - loss: 0.0046 - val_loss: 0.0058 - 332ms/epoch - 9ms/step
Ep

Epoch 14/200
35/35 - 0s - loss: 0.0062 - val_loss: 0.0084 - 284ms/epoch - 8ms/step
Epoch 15/200
35/35 - 0s - loss: 0.0061 - val_loss: 0.0087 - 262ms/epoch - 7ms/step
Epoch 16/200
35/35 - 0s - loss: 0.0060 - val_loss: 0.0081 - 285ms/epoch - 8ms/step
Epoch 17/200
35/35 - 0s - loss: 0.0060 - val_loss: 0.0079 - 272ms/epoch - 8ms/step
Epoch 18/200
35/35 - 0s - loss: 0.0059 - val_loss: 0.0079 - 266ms/epoch - 8ms/step
Epoch 19/200
35/35 - 0s - loss: 0.0058 - val_loss: 0.0079 - 276ms/epoch - 8ms/step
Epoch 20/200
35/35 - 0s - loss: 0.0057 - val_loss: 0.0076 - 281ms/epoch - 8ms/step
Epoch 21/200
35/35 - 0s - loss: 0.0056 - val_loss: 0.0074 - 262ms/epoch - 7ms/step
Epoch 22/200
35/35 - 0s - loss: 0.0055 - val_loss: 0.0071 - 276ms/epoch - 8ms/step
Epoch 23/200
35/35 - 0s - loss: 0.0054 - val_loss: 0.0071 - 255ms/epoch - 7ms/step
Epoch 24/200
35/35 - 0s - loss: 0.0053 - val_loss: 0.0071 - 257ms/epoch - 7ms/step
Epoch 25/200
35/35 - 0s - loss: 0.0052 - val_loss: 0.0066 - 260ms/epoch - 7ms/step
Epoc

Epoch 113/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0058 - 328ms/epoch - 9ms/step
Epoch 114/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0055 - 289ms/epoch - 8ms/step
Epoch 115/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0057 - 285ms/epoch - 8ms/step
Epoch 116/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0057 - 240ms/epoch - 7ms/step
Epoch 117/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0057 - 254ms/epoch - 7ms/step
Epoch 118/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0059 - 268ms/epoch - 8ms/step
Epoch 119/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0059 - 288ms/epoch - 8ms/step
Epoch 120/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0056 - 282ms/epoch - 8ms/step
Epoch 121/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0061 - 244ms/epoch - 7ms/step
Epoch 122/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0056 - 305ms/epoch - 9ms/step
Epoch 123/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0057 - 272ms/epoch - 8ms/step
Epoch 124/200
35/35 - 0s - loss: 0.0042 - val_loss: 0.0056 - 290ms/epoch - 8

70/70 - 6s - loss: 0.0762 - val_loss: 0.0951 - 6s/epoch - 83ms/step
Epoch 2/100
70/70 - 0s - loss: 0.0652 - val_loss: 0.0746 - 468ms/epoch - 7ms/step
Epoch 3/100
70/70 - 0s - loss: 0.0507 - val_loss: 0.0543 - 429ms/epoch - 6ms/step
Epoch 4/100
70/70 - 0s - loss: 0.0381 - val_loss: 0.0420 - 411ms/epoch - 6ms/step
Epoch 5/100
70/70 - 0s - loss: 0.0308 - val_loss: 0.0372 - 426ms/epoch - 6ms/step
Epoch 6/100
70/70 - 0s - loss: 0.0281 - val_loss: 0.0350 - 432ms/epoch - 6ms/step
Epoch 7/100
70/70 - 1s - loss: 0.0273 - val_loss: 0.0347 - 546ms/epoch - 8ms/step
Epoch 8/100
70/70 - 0s - loss: 0.0271 - val_loss: 0.0337 - 460ms/epoch - 7ms/step
Epoch 9/100
70/70 - 0s - loss: 0.0269 - val_loss: 0.0331 - 364ms/epoch - 5ms/step
Epoch 10/100
70/70 - 0s - loss: 0.0269 - val_loss: 0.0342 - 335ms/epoch - 5ms/step
Epoch 11/100
70/70 - 0s - loss: 0.0269 - val_loss: 0.0348 - 318ms/epoch - 5ms/step
Epoch 12/100
70/70 - 0s - loss: 0.0268 - val_loss: 0.0344 - 317ms/epoch - 5ms/step
Epoch 13/100
70/70 - 0s - l

109/109 - 1s - loss: 0.0021 - val_loss: 0.0088 - 671ms/epoch - 6ms/step
Epoch 43/100
109/109 - 1s - loss: 0.0021 - val_loss: 0.0088 - 658ms/epoch - 6ms/step
Epoch 44/100
109/109 - 1s - loss: 0.0021 - val_loss: 0.0087 - 656ms/epoch - 6ms/step
Epoch 45/100
109/109 - 1s - loss: 0.0021 - val_loss: 0.0087 - 683ms/epoch - 6ms/step
Epoch 46/100
109/109 - 1s - loss: 0.0020 - val_loss: 0.0087 - 704ms/epoch - 6ms/step
Epoch 47/100
109/109 - 1s - loss: 0.0020 - val_loss: 0.0085 - 731ms/epoch - 7ms/step
Epoch 48/100
109/109 - 1s - loss: 0.0019 - val_loss: 0.0084 - 698ms/epoch - 6ms/step
Epoch 49/100
109/109 - 1s - loss: 0.0019 - val_loss: 0.0083 - 759ms/epoch - 7ms/step
Epoch 50/100
109/109 - 1s - loss: 0.0018 - val_loss: 0.0080 - 738ms/epoch - 7ms/step
Epoch 51/100
109/109 - 1s - loss: 0.0018 - val_loss: 0.0082 - 742ms/epoch - 7ms/step
Epoch 52/100
109/109 - 1s - loss: 0.0017 - val_loss: 0.0077 - 711ms/epoch - 7ms/step
Epoch 53/100
109/109 - 1s - loss: 0.0017 - val_loss: 0.0076 - 663ms/epoch - 6m

## CONV-LSTMATTNAE

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-08-24 10:38:06.419496: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-24 10:38:06.419559: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [4]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [ ]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Conv_LstmAttnAE_HYP.csv')


input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_test =  {}
source_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = Conv_LstmAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_Conv_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_Conv_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = train_merge_df.reshape(train_merge_df.shape[0], 1, train_merge_df.shape[1])

model = Conv_LstmAttnAEModel(layer1=40, layer2=40, encoding_dim=2)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_test =  {}
mtl_dict_results_train = {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = xtrain.reshape(xtrain.shape[0], 1, xtrain.shape[1])
    xtest = xtest.reshape(xtest.shape[0], 1, xtest.shape[1])
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_Conv_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_Conv_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

2023-08-14 22:17:07.095881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-14 22:17:07.096037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-08-14 22:17:07.096168: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-08-14 22:17:07.096255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-08-14 22:17:07.130696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Epoch 1/100
55/55 - 11s - loss: 0.0543 - val_loss: 0.0544 - 11s/epoch - 196ms/step
Epoch 2/100
55/55 - 6s - loss: 0.0234 - val_loss: 0.0503 - 6s/epoch - 113ms/step
Epoch 3/100
55/55 - 6s - loss: 0.0221 - val_loss: 0.0494 - 6s/epoch - 111ms/step
Epoch 4/100
55/55 - 6s - loss: 0.0212 - val_loss: 0.0484 - 6s/epoch - 110ms/step
Epoch 5/100
55/55 - 6s - loss: 0.0201 - val_loss: 0.0477 - 6s/epoch - 110ms/step
Epoch 6/100
55/55 - 6s - loss: 0.0193 - val_loss: 0.0467 - 6s/epoch - 111ms/step
Epoch 7/100
55/55 - 6s - loss: 0.0183 - val_loss: 0.0448 - 6s/epoch - 111ms/step
Epoch 8/100
55/55 - 6s - loss: 0.0172 - val_loss: 0.0439 - 6s/epoch - 110ms/step
Epoch 9/100
55/55 - 6s - loss: 0.0163 - val_loss: 0.0426 - 6s/epoch - 111ms/step
Epoch 10/100
55/55 - 6s - loss: 0.0160 - val_loss: 0.0416 - 6s/epoch - 111ms/step
Epoch 11/100
55/55 - 6s - loss: 0.0156 - val_loss: 0.0410 - 6s/epoch - 112ms/step
Epoch 12/100
55/55 - 6s - loss: 0.0155 - val_loss: 0.0406 - 6s/epoch - 112ms/step
Epoch 13/100
55/55 - 6s